- Windows10 64bit
- Python3.5
- MongoDB 4.0.2

参考

- [『Pythonクローリング＆スクレイピング―データ収集・解析のための実践開発ガイド―』技術評論社](https://gihyo.jp/book/2017/978-4-7741-8367-1)
- [『退屈なことはPythonにやらせよう――ノンプログラマーにもできる自動化処理プログラミング』O'Reilly Japan](https://www.oreilly.co.jp/books/9784873117782/)

# PythonからMongoDBを使用

MongoDB用のディレクトリを作成しておく

バックグラウンドでMongoDBを起動しておく

起動

`mongod --dbpath <MongoDBのデータ用ディレクトリへのパス>`


参考：http://kageura.hatenadiary.jp/entry/2018/01/09/Windows%E7%89%88MongoDB%E3%81%AE%E3%82%A4%E3%83%B3%E3%82%B9%E3%83%88%E3%83%BC%E3%83%AB%E3%83%BBMongoShell%E3%82%92%E9%80%9A%E3%81%97%E3%81%A6CRUD%E3%82%B3%E3%83%9E%E3%83%B3%E3%83%89%E3%82%92%E6%89%93

`BulkWriteError: batch op errors occurred`

でよく止まる

→動画が存在しない部分まで検索対象としてしまっている可能性がある。

In [2]:
import os
import sys

from apiclient.discovery import build # pip install google-api-python-client
from pymongo import MongoClient, DESCENDING

YOUTUBE_API_KEY = '<APIキーを記入>' # 環境変数（.envファイル）からAPIを呼び出したいが、JupyterNotebookからだとなぜかエラーになる。

query = '輝夜月'
max_pages=8
maxResults=50

def main():
    """
    メイン処理
    """
    mongo_client = MongoClient('localhost', 27017)
    db = mongo_client.kaguya2
    collection = db.luna # kaguyaデータベース -> lunaコレクション
    collection.delete_many({}) # 既存の全てのドキュメントを削除

    sort_key='statistics.viewCount' # ランキング基準の指定

    for items_per_page in search_videos(query, max_pages, maxResults):
        save_to_mongodb(collection, items_per_page)


def search_videos(query, max_pages=10, maxResults=50):
    """
    動画を検索してページ単位でlistをyieldする
    """
    youtube = build('youtube', 'v3', developerKey = YOUTUBE_API_KEY)

    search_request = youtube.search().list(
        part='id',
        q=query,
        type='video',
        maxResults=maxResults,
    )
    
    i = 0
    while search_request and i < max_pages:
        search_response = search_request.execute() # execute()で実際にHTTPリクエストを送信。APIのレスポンスを取得。
        video_ids = [item['id']['videoId'] for item in search_response['items']]

        videos_response = youtube.videos().list(
            part='snippet,statistics',
            id=','.join(video_ids)
        ).execute()

        yield videos_response['items']

        search_request = youtube.search().list_next(search_request, search_response)
        i += 1


def save_to_mongodb(collection, items):
    """
    MongoDBにアイテムのリストを保存
    """
    for item in items:
        item['_id'] = item['id']

        for key, value in item['statistics'].items():
            item['statistics'][key] = int(value)

    result = collection.insert_many(items) # コレクションに挿入
    print('Inserted {0} documents'.format(len(result.inserted_ids)), file=sys.stderr)


if __name__ == '__main__':
    main()


Inserted 50 documents
Inserted 50 documents
Inserted 50 documents
Inserted 50 documents
Inserted 50 documents
Inserted 50 documents
Inserted 50 documents
Inserted 50 documents


# 整形してMongoDBから抽出したデータをcsvに保存 

タイトル、URL、チャンネル名、公開日、視聴回数、Like数、Dislike数、コメント数（一部不正確？）、favorite数（不明）

In [3]:
import csv

mongo_client = MongoClient('localhost', 27017)
db = mongo_client.kaguya
collection = db.luna # kaguyaデータベース -> lunaコレクション

# 列名（1行目）を作成
## [タイトル、URL、チャンネル名、公開日、視聴回数、Like数、Dislike数、コメント数（一部不正確？）、favorite数（不明）]
col_name = ['title', 'url', 'channelTitle', 'publishedAt']
statistics_keys = ['viewCount', 'likeCount', 'dislikeCount', 'commentCount', 'favoriteCount']
col_name.extend(statistics_keys)
sort_key = 'statistics.likeCount' # ひとまずLikeが多い順に保存

with open('youtube_result.csv', 'w', newline='', encoding='utf-8') as output_csv:
    csv_writer = csv.writer(output_csv)
    csv_writer.writerow(col_name) # 列名を記入
    
    # データを整形しつつcsvに書き込んでいく
    for item in collection.find().sort(sort_key, DESCENDING):
        url = 'https://www.youtube.com/watch?v=' + item['_id']
        row_items = [item['snippet']['title'], url, item['snippet']['channelTitle'], item['snippet']['publishedAt']]
        
        # 値が入っていない部分を埋めるために統計量についてfor文を回す
        for statistics_key in statistics_keys:
            item['statistics'].setdefault(statistics_key, 0)
            row_items.append(item['statistics'][statistics_key]) # 列名と同じ順番になるようにstatistics_keysでfor文を回す

        csv_writer.writerow(row_items)


In [4]:
# 輝夜月の公式動画のみ抽出
for item in collection.find({'snippet.channelTitle': 'Kaguya Luna Official'}).sort('statistics.viewCount', DESCENDING):
    print(item['statistics']['viewCount'], item['snippet']['channelTitle'], item['snippet']['title'])

3078789 Kaguya Luna Official 【Getting Over It】月ちゃんおこだよ！！！！！おこ
2949664 Kaguya Luna Official 【自己紹介】輝夜 月の特技がスゴイ！！！！
2677585 Kaguya Luna Official 必殺！あいさつきめてみたっス！！！！！！！！！！！！！！！！！！
2469026 Kaguya Luna Official 老化防止対策してないとかマ？！
2079630 Kaguya Luna Official 【声真似】誰が一番月ちゃんに似てるか選手権ｗｗｗｗ
1866509 Kaguya Luna Official 【バーチャルユーチューバー】初コラボまじクソワロタｗｗｗｗｗｗ
1774868 Kaguya Luna Official 質問答えたったワｗｗｗｗｗｗｗ
1737377 Kaguya Luna Official あけおめワッショイ∠ 'ω'／
1726593 Kaguya Luna Official 【受験生必見】声ってマジ大事だよ～って⤴⤴
1661631 Kaguya Luna Official 【声真似】バーチャルユーチューバーのガチ声真似しましたまる
1651863 Kaguya Luna Official 女子MJK！！！
1452977 Kaguya Luna Official 【クリスマス】え・・・・まさかおまえらって・・・・・・
1385093 Kaguya Luna Official Twitterが面白くなる方法をみつけたんだがｗｗｗｗｗｗ
1339319 Kaguya Luna Official 【エイプリルフール】これが月達の嘘でっす！！！！！、、！！！、
1331167 Kaguya Luna Official 【血液型】月ちゃん特技極めたったワ
1312194 Kaguya Luna Official 【作業用BGM】スト●グゼ●飲んだらこんな感じｗｗｗｗｗｗｗｗｗｗｗ
1290848 Kaguya Luna Official 【バレンタイン】もしも月ちゃんが彼女だったらの動画
1262064 Kaguya Luna Official 【悲報】俺はおこている！！！！！！！！！、、！！！、
1261624 Kaguya Luna Officia

# 保存したデータをpandasに読み込んで分析

In [5]:
import pandas as pd

# csvファイルを読み込む
df = pd.read_csv('youtube_result.csv')

# 「活発度」列を作る
# 再生数に対してどれだけのコメントが集まっているか
df['活発度'] = df['commentCount'] / df['viewCount']

# 「好感度」列を作る
# 再生数に対してどれだけLikeが集まっているか
df['好感度'] = df['likeCount'] / df['viewCount']

In [6]:
# 好感度順にソート
# オフィシャル動画のみ抽出
df[df['channelTitle'] == 'Kaguya Luna Official'].sort_values(by=['好感度'], ascending=False)

,title,url,channelTitle,publishedAt,viewCount,likeCount,dislikeCount,commentCount,favoriteCount,活発度,好感度
40,【むかしむかしゲーム】くそおもろいｗｗｗｗｗｗｗ【Twitter】,https://www.youtube.com/watch?v=F9hW7PdE510,Kaguya Luna Official,2018-08-10T10:00:14.000Z,407125,20844,321,1343,0,0.003299,0.051198
50,【安眠動画】読み聞かせして寝させるっていう動画ッスｗｗｗｗ,https://www.youtube.com/watch?v=5OZPn6_NIuE,Kaguya Luna Official,2018-08-29T15:02:43.000Z,342761,15802,232,3147,0,0.009181,0.046102
2,【怒ってる？】べ、べべべ、べべべイベベベイベー【輝夜月風】,https://www.youtube.com/watch?v=u-Amlu3jc3U,Kaguya Luna Official,2018-06-24T11:00:08.000Z,1180912,53780,945,4504,0,0.003814,0.045541
24,【縦動画】Hey Luna!【スマホ推奨】,https://www.youtube.com/watch?v=bUeiFSjLQl4,Kaguya Luna Official,2018-07-09T11:00:05.000Z,675881,29713,363,2342,0,0.003465,0.043962
43,【世界初】何も始まってません【初ホラーゲーム実況】,https://www.youtube.com/watch?v=jNB3LA6rlAI,Kaguya Luna Official,2018-08-16T11:00:11.000Z,481190,19212,279,2371,0,0.004927,0.039926
22,【天使と悪魔】月の脳内会議ｗｗｗｗｗｗｗｗｗｗ,https://www.youtube.com/watch?v=j60sNy_kqg0,Kaguya Luna Official,2018-05-08T12:30:01.000Z,800871,31342,348,3352,0,0.004185,0.039135
52,【炎上】アーチーチーアーチー！ 燃えてるんだろうかァァァァ,https://www.youtube.com/watch?v=SCSCHrQUaag,Kaguya Luna Official,2018-08-22T11:00:19.000Z,392644,14854,219,5114,0,0.013025,0.037831
53,【3分でわかる】輝夜月生放送,https://www.youtube.com/watch?v=rwcbJIG3dug,Kaguya Luna Official,2018-08-26T11:00:35.000Z,357371,13495,224,1691,0,0.004732,0.037762
30,【綺麗な動画】YouTube１きれいな動画です(たぶん),https://www.youtube.com/watch?v=IMhtEOYCg4Y,Kaguya Luna Official,2018-06-16T12:30:00.000Z,684396,25510,713,2524,0,0.003688,0.037274
58,輝夜 月VR LIVE@Zepp VR第一弾ティザー映像,https://www.youtube.com/watch?v=mP3My5vIoTs,Kaguya Luna Official,2018-08-11T08:00:02.000Z,265955,9841,79,873,0,0.003283,0.037003


参考

In [7]:
# 視聴回数上位100位で活発度順にソート
df.sort_values(by=['viewCount'], ascending=False).head(100).sort_values(by=['活発度'], ascending=False)

,title,url,channelTitle,publishedAt,viewCount,likeCount,dislikeCount,commentCount,favoriteCount,活発度,好感度
52,【炎上】アーチーチーアーチー！ 燃えてるんだろうかァァァァ,https://www.youtube.com/watch?v=SCSCHrQUaag,Kaguya Luna Official,2018-08-22T11:00:19.000Z,392644,14854,219,5114,0,0.013025,0.037831
50,【安眠動画】読み聞かせして寝させるっていう動画ッスｗｗｗｗ,https://www.youtube.com/watch?v=5OZPn6_NIuE,Kaguya Luna Official,2018-08-29T15:02:43.000Z,342761,15802,232,3147,0,0.009181,0.046102
70,【描いて見た】「#キズナアイ 10分チャレンジ」に挑戦！,https://www.youtube.com/watch?v=CXq_I476A6Y,Siro Channel,2018-06-22T10:00:55.000Z,110835,4407,55,775,0,0.006992,0.039762
91,輝夜月とミライアカリを入れかえてみた！Kaguya Luna and Mirai Akari...,https://www.youtube.com/watch?v=Hduso_k6Ba0,ユーチューバーが好きすぎてどうにかなっちゃいそう,2018-05-12T09:52:13.000Z,30332,761,6,207,0,0.006824,0.025089
65,【初投稿】パンツが設定画にのっていたことについての釈明会見,https://www.youtube.com/watch?v=-6OsvuwI0Ck,HimeHina Channel,2018-03-22T12:00:00.000Z,140532,5241,75,938,0,0.006675,0.037294
19,【安眠動画】おやすみなさい、いい夢みれますようｗｗｗにｗｗｗｗ,https://www.youtube.com/watch?v=SspwaZK8h7k,Kaguya Luna Official,2018-06-05T12:00:05.000Z,1022238,34421,1427,6817,0,0.006669,0.033672
64,【急上昇バーチャルYouTubeｒ】２Ｄの俺氏惨敗ｗ人気の秘訣に迫る,https://www.youtube.com/watch?v=EoXnR0XoSqA,たっくーTVれいでぃお,2017-12-28T14:59:28.000Z,371247,6078,188,2396,0,0.006454,0.016372
84,VTuberの輝夜月ちゃんについて語らせてくれ,https://www.youtube.com/watch?v=KMpLhzX9PSY,織田信姫,2018-06-02T09:00:01.000Z,34162,1355,63,220,0,0.006440,0.039664
56,【生放送】バーチャルYouTuberになってVR世界を荒らしたったｗｗｗｗｗ【キズナアイ.輝夜月】,https://www.youtube.com/watch?v=5Ma_WZZSZPQ,コレコレチャンネル KoreTube,2018-01-10T11:49:35.000Z,408411,11760,1890,2506,0,0.006136,0.028795
37,"【Quick,Draw!】月ちゃんお絵描き上手い？下手？？【描いてみた】",https://www.youtube.com/watch?v=eUTg5YbDMKI,Kaguya Luna Official,2018-04-15T13:00:03.000Z,923196,23119,330,5280,0,0.005719,0.025042
